# oobabooga/text-generation-webui
1. docker pull c00cjz00/pytorch:2.1.0-cuda11.8-cudnn8-runtime_textgen
2. 01_create_image_ipykernel.ipynb

In [ ]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path
current_foldr=!pwd
current_foldr=current_foldr[0]
current_foldr

### 安裝 library

In [ ]:
pip install rich -q

## 主程式 
1. 請修改 倒數第五行 password="nchc:nchcorgtw" 

In [ ]:
%%bash
#  DOWNLOAD text-generation-webui, install requirements.txt and execute this service 

REPOSRC=https://github.com/oobabooga/text-generation-webui.git
LOCALREPO=text-generation-webui
BRANCH="snapshot-2023-12-10"
# We do it this way so that we can abstract if from just git later on
LOCALREPO_VC_DIR=$LOCALREPO/.git

if [ ! -d $LOCALREPO_VC_DIR ]
then
    git clone --branch $BRANCH $REPOSRC $LOCALREPO

    # It does not need to pip with this IMAGE
    #!pip install -r text-generation-webui/temp_requirements.txt -q 

    # Prepare
    cp ./textgen/config-user.yaml ./text-generation-webui/models/config-user.yaml
    cp ./textgen/server_nchc.py ./text-generation-webui/server_nchc.py
    cd ./text-generation-webui/models
    ln -s /work/u00cjz00/slurm_jobs/github/models/Taiwan-LLM-7B-v2.1-chat-Q8_0.gguf .
    ln -s /work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf .
    ln -s /work/u00cjz00/slurm_jobs/github/models/Llama-2-7B-Chat-GPTQ .
    cd ../../
    cd ./text-generation-webui/training/datasets
    ln -s /work/u00cjz00/slurm_jobs/github/dataset/school_math_1000.json .
    ln -s /work/u00cjz00/slurm_jobs/github/dataset/school_math_30000.json .
    cd ../../../
    cd ./text-generation-webui/loras
    ln -s /work/u00cjz00/slurm_jobs/github/loras/math .
    cd ../../
    
fi 

# GET IP and Free Port
# IP
node_ip=$(cat /etc/hosts |grep "$(hostname -a)" | awk '{print $1}')
# PORT
noed_port_genai=$(python -c "import socket; s = socket.socket(socket.AF_INET, socket.SOCK_STREAM); s.bind(('', 0)); addr = s.getsockname(); s.close(); print(addr[1])")
noed_port_genai=44727
# PROXY
proxy_url=/rstudio/${node_ip}/${noed_port_genai}
# URL
https_url=https://node01.biobank.org.tw${proxy_url}/
echo $https_url

# 強制刪除運行中服務
ps -ef |grep server.py | awk '{print $2}' | xargs kill -9

# EXPORT PATH for ~/.local
export PATH=$PATH:$HOME/.local/bin

# 主程式
password="nchc:nchcorgtw"
cd text-generation-webui

HF_TOKEN='hf_xxxxx' python server_nchc.py  --listen --listen-port ${noed_port_genai} --listen-host $(hostname -s) \
--chat-buttons --ssl-keyfile ${proxy_url} --gradio-auth ${password} --extensions Training_PRO

#HF_TOKEN='hf_xxxxx' python server.py --listen --listen-port ${noed_port_genai} --listen-host $(hostname -s) \
#--chat-buttons --gradio-auth ${password} --extensions Training_PRO

In [ ]:
# 強制刪除服務
!ps -ef |grep server.py | awk '{print $2}' | xargs kill -9